In [2]:
import cv2
from imutils.object_detection import non_max_suppression # pip install imutils
from loguru import logger
import numpy as np

In [15]:
!pip install imutils

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [22]:
img_path = '/tmp/sanrezhen.jpg'
template_path = '/tmp/sanrezhen_template.jpg'
template_path = '/Users/xbkaishui/opensource/cv_hz/app_backend/backend/app/app/tests/api/api_v1/sanrezhen_template2.jpg'
template_path = '/Users/xbkaishui/opensource/cv_hz/app_backend/backend/app/app/tests/api/api_v1/sanrezhen_template.jpg'
  # Load the image and template
image = cv2.imread(img_path, cv2.IMREAD_COLOR)
template = cv2.imread(template_path, cv2.IMREAD_COLOR)
  
  # Perform template matching 
result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
  
  # Filter results to only good ones
threshold = 0.4 # Larger values have less, but better matches.
(yCoords, xCoords) = np.where(result >= threshold)

print(len(yCoords))
# sort by score

# Perform non-maximum suppression.
template_h, template_w = template.shape[:2]
rects = []
probs = []
for (x, y) in zip(xCoords, yCoords):
    rects.append((x, y, x + template_w, y + template_h))
    probs.append(result[y, x])
pick = non_max_suppression(np.array(rects), probs=np.array(probs), overlapThresh=.4)

# Optional: Visualize the results
for (startX, startY, endX, endY) in pick:
    cv2.rectangle(image, (startX, startY), (endX, endY),(0, 255, 0), 2)
cv2.imwrite('/tmp/sanrezhen_result.jpg', image)
cv2.imshow('Results', image)

12986


In [3]:
img_path = '/tmp/sanrezhen.jpg'
template_path = '/tmp/sanrezhen_template.jpg'
  # Load the image and template
image = cv2.imread(img_path, cv2.IMREAD_COLOR)
template = cv2.imread(template_path, cv2.IMREAD_COLOR)
  
# Perform template matching 
result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

result2 = np.reshape(result, result.shape[0]*result.shape[1])
sort = np.argsort(result2)
(y,x) = np.unravel_index(sort, result.shape)
print(y.shape)


(3230136,)


In [26]:
# test performance
import time
img_path = '/tmp/sanrezhen.jpg'
template_path = '/tmp/sanrezhen_template.jpg'
template_path = '/Users/xbkaishui/opensource/cv_hz/app_backend/backend/app/app/tests/api/api_v1/sanrezhen_template2.jpg'
template_path = '/Users/xbkaishui/opensource/cv_hz/app_backend/backend/app/app/tests/api/api_v1/sanrezhen_template.jpg'
  # Load the image and template
image = cv2.imread(img_path, cv2.IMREAD_COLOR)
template = cv2.imread(template_path, cv2.IMREAD_COLOR)

start = time.time()
count = 100
for i in range(count):
  # Perform template matching 
    result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    
    # Filter results to only good ones
    threshold = 0.4 # Larger values have less, but better matches.
    (yCoords, xCoords) = np.where(result >= threshold)

    # print(len(yCoords))
    # sort by score

    # Perform non-maximum suppression.
    template_h, template_w = template.shape[:2]
    rects = []
    probs = []
    for (x, y) in zip(xCoords, yCoords):
        rects.append((x, y, x + template_w, y + template_h))
        probs.append(result[y, x])
    pick = non_max_suppression(np.array(rects), probs=np.array(probs), overlapThresh=.4)

end = time.time()
logger.info("total cost {}", (end-start)*1000)

# Optional: Visualize the results
for (startX, startY, endX, endY) in pick:
    cv2.rectangle(image, (startX, startY), (endX, endY),(0, 255, 0), 2)
cv2.imwrite('/tmp/sanrezhen_result.jpg', image)
cv2.imshow('Results', image)

2023-12-25 09:10:06.952 | INFO     | __main__:<module>:34 - total cost 22848.506927490234


In [27]:
def rotate_image(image, angle):
    height, width = image.shape[:2]
    image_center = (width / 2, height / 2)
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    abs_cos = abs(rotation_mat[0, 0])
    abs_sin = abs(rotation_mat[0, 1])
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)
    rotation_mat[0, 2] += bound_w / 2 - image_center[0]
    rotation_mat[1, 2] += bound_h / 2 - image_center[1]
    rotated_image = cv2.warpAffine(image, rotation_mat, (bound_w, bound_h), borderMode=cv2.BORDER_REPLICATE)
    return rotated_image

In [28]:
# test rotation 
template_path = '/tmp/sanrezhen_template.jpg'
img = cv2.imread(template_path)
rot_img = rotate_image(img, 30)
print(rot_img.shape)
cv2.imwrite('/tmp/sanrezhen_template_rot.jpg', rot_img)

(257, 229, 3)


True